In [1]:
from Paciente import Paciente

In [2]:
import numpy as np
from scipy.ndimage import label, center_of_mass, sum  
import pandas as pd

In [3]:
# Se instancia la clase para el paciente numero 29
paciente29 = Paciente(29, 3)

No damaged files


Construyendo Mascaras:   9%|▉         | 32/348 [00:02<00:09, 33.11it/s]

KeyboardInterrupt: 

Construyendo Mascaras:  10%|█         | 35/348 [00:20<00:09, 33.11it/s]

In [ ]:
pat_height = paciente29.altura
pat_weight = paciente29.peso
pat_BSA = np.sqrt((pat_height*pat_weight)/3600) #m^2?

In [ ]:
spacing = paciente29.spacing
data = np.array(paciente29.mascaraGeneral)

# Separate the different lesions and give them different labels
# Not connected regions will be seen as different objects 
labeled_data, num_features = label(data)
print('Number of lesions detected: ' + str(num_features))

seg = np.argwhere(labeled_data)
centr = np.asarray(center_of_mass(data, labels = labeled_data,
                                    index = list(range(1, num_features+1))))

In [ ]:
def weighted_distance(p1, p2, weights, type_of_distance):
    try: 
        len(weights) == len(p1) == len(p2)
    except: 
        print('Error: Dimension mismatch between data points and weight array') 
    else: 
        # For every type of distance there is a formula for a weighted distance
        # which will be executed if the corresponding type is selected. 
        
        q = p1 - p2 
        
        if type_of_distance == 'euclidian':
            # Weighted Euclidian distance
            return np.sqrt(((weights * q)**2).sum())
        
        elif type_of_distance == 'manhattan' or type_of_distance == 'cityblock':
            # Weighted Manhattan or Cityblock distance 
            return (weights * abs(q)).sum()
            
        elif type_of_distance == 'chebyshev':
            # Weighted Chebyshev distance 
            return max(weights * abs(q))
            
        else: 
            # If a different string than the possible distance types is given, 
            # the following statement is printed. 
            print("Error: The selected distance type is not available. \
                    Try 'euclidian', 'manhattan' or 'cityblock', or 'chebyshev'.")


def distances(segmentation, weights):
    euc_dist = [weighted_distance(segmentation[i], segmentation[j], weights, 'euclidian')
                for i in range(len(segmentation)) for j in range(len(segmentation)) if i != j]
    man_dist = [weighted_distance(segmentation[i], segmentation[j], weights, 'manhattan')
                for i in range(len(segmentation)) for j in range(len(segmentation)) if i != j]
    che_dist = [weighted_distance(segmentation[i], segmentation[j], weights, 'chebyshev')
                for i in range(len(segmentation)) for j in range(len(segmentation)) if i != j]
    index_i = [i for i in range(len(segmentation)) for j in range(len(segmentation)) if i != j]

    return euc_dist, man_dist, che_dist, index_i


In [ ]:
# Distances between all pairs of centroids are calculated 
euc_dist_ctr, man_dist_ctr, che_dist_ctr, index_i_ctr = distances(centr, spacing)
dist_ctr = np.array([euc_dist_ctr, man_dist_ctr, che_dist_ctr, index_i_ctr])

In [ ]:
# Function that gives back all the euclidian distances between one specific 
# lesion and all the others 
def lesion_distances(index): 
    # Give back the array of euclidian distances between one specific lesion and the others
    d = dist_ctr[0, np.where(dist_ctr[3] == index)[0]]
    return d

In [ ]:
#%% Calculate parameters

# Calculation of parameters, based on centroid distances 
# Dmax_patient: (Euclidian) Distance between the two lesions that are the farthest away from each other. 
Dmax_patient = max(euc_dist_ctr)

# Dmax_bulk: (Euclidian) Distance between the largest lesion and the one the farthest away from it. 
# Find largest lesion first 
sizes_of_lesions = [sum(data, labeled_data, index = i)
                    for i in range(1, num_features+1)]
largest_size = max(sizes_of_lesions) # how many pixels does the largest lesion have?
largest_index = np.where(sizes_of_lesions == largest_size)[0][0] # which Python Index does the largest lesion have?
largest_label = largest_index + 1 # which Label does the largest lesion have?  
#print('The largest lesion has label index ' + str(largest_label))
# Calculate the parameter
Dmax_bulk = max(lesion_distances(largest_index))

# SPREAD_bulk: Sum of the euclidian distances between the largest lesion and all the other lesions.
Spread_bulk = np.sum(lesion_distances(largest_index))

# SPREAD_patient: Over all lesions, the maximum of the sum of distances from one lesion to all the others.
sums = [np.sum(lesion_distances(i)) for i in range(num_features)]
Spread_patient = max(sums)

# SDmax_euc: Dmax_patient, normalized by BSA
SDmax_euc = Dmax_patient/pat_BSA

# SDmax_man: Maximum Manhattan distance between lesions, normalized by BSA. 
SDmax_man = max(man_dist_ctr)/pat_BSA

# SDmax_che: Maximum Chebyshev distance between lesions, normalized by BSA. 
SDmax_che = max(che_dist_ctr)/pat_BSA

#%% Add important features to Results file (output)
results_dict = {
    'Height': [pat_height],
    'Weight': [pat_weight],
    'BSA': [pat_BSA],
    'Dmax_patient': [Dmax_patient],
    'Dmax_bulk': [Dmax_bulk],
    'Spread_bulk': [Spread_bulk],
    'Spread_patient': [Spread_patient],
    'SDmax_euc': [SDmax_euc],
    'SDmax_man': [SDmax_man],
    'SDmax_che': [SDmax_che]
}

# Convertir el diccionario en un DataFrame
results_df = pd.DataFrame(results_dict)

# Distancias en milimetros
results_df